In [1]:
import os
import sys
import numpy as np
import nibabel as nib
import pydicom
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from numpy import zeros, newaxis
from scipy.stats import norm
import cv2
import scipy.signal as signal
from pydicom.pixel_data_handlers.util import apply_color_lut
from skimage.filters import threshold_multiotsu
from skimage.filters import threshold_otsu, threshold_local
import scipy.io
import pandas as pd
import SimpleITK as sitk

# Util functions

In [228]:
def plot_data(data, axis=0):
    if axis==0:
        for i in range(0, len(data[0,0,:])):
            img=data[:,:,i]
            plt.imshow(img)
            plt.show()
    elif axis==1:
        for i in range(0, len(data[0,:,0])):
            img=data[:,i,:]
            plt.imshow(img)
            plt.show()
    elif axis==2:
        for i in range(0, len(data[:,0,0])):
            img=data[i,:,:]
            plt.imshow(img)
            plt.show()
            
def ch_coordinates(bottom, top, nchannels, vox_res=39.06):
#     ch_coord=np.linspace(pstart, pstop, nchannels).astype('int')
    unitvec=top-bottom
    unitvec=unitvec/np.linalg.norm(unitvec)
    
    ch_coord=np.zeros((nchannels,3))
    for i in range(nchannels):
        ch_coord[i,:]=np.round(bottom+(i*unitvec*50/vox_res))
    
    return (ch_coord).astype('int'), unitvec

def fixed2movingPoints(movingPath, fixedPath, transformPath, fixedPointsSave, movingPointsSave):
    try:
        transform_moving2fixed=sitk.ReadTransform(transformPath)
        print("Succesfully loaded transform: "+transformPath.split("/")[-1])
    except:
        print("error loading the transform file")
        return
    
    try:
        movingImg=sitk.ReadImage(movingPath)  # Moving image
        print("Succesfully loaded moving img: "+movingPath.split("/")[-1])
    except:
        print("error loading the moving img file")
        return
    
    try:
        fixedImg=sitk.ReadImage(fixedPath)  # Fixed image
        print("Succesfully loaded fixed img: "+fixedPath.split("/")[-1])
    except:
        print("error loading the fixed img file")
        return
    
    nx, ny, nz= fixedImg.GetSize()
    
    moving_coordinates=np.empty((nx*ny*nz, 3))
    fixed_coordinates=np.empty((nx*ny*nz, 3))
    
    
    i=0
    for x in range(nx):
        if x%10==0:
            print("Progress: " + str(x) + "/" +str(nx))
        for y in range(ny):
            for z in range(nz):
                idx=[x,y,z]
                fixedpnt=fixedImg.TransformIndexToPhysicalPoint(idx)
                movingpnt=transform_moving2fixed.TransformPoint(fixedpnt)
                movingidx=movingImg.TransformPhysicalPointToIndex(movingpnt)
                fixed_coordinates[i,:]=idx
                moving_coordinates[i,:]=movingidx
                i=i+1
    
#     fixedcorr_path="/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/fixed_img-idx.npy"
    np.save(fixedPointsSave, fixed_coordinates)

#     movingcorr_path="/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/moving_presurgeryresampled_img-idx.npy"
    np.save(movingPointsSave, moving_coordinates)
    
    return fixed_coordinates, moving_coordinates

## Reading atlas labels

In [183]:
labels_path='./atlas_labels.rtf'
labels=[]
anats=[]
with open(labels_path) as f:
    lines = f.readlines()
    for i,line in enumerate(lines):
        if i>6 and i%2==1:
            if i==7:
                anat= ' '.join(line.split('\\cf')[1].split()[8:])
#                 label=[line.split('\\cf')[1].split()[0], anat]
                anats.append(anat.split("\"")[1])
                labels.append(int(line.split('\\cf')[1].split()[0]))
#                 print(label)
            else:
                anat=' '.join(line.split()[7:])
#                 label=[line.split()[0], anat]
                anats.append(anat.split("\"")[1])
                labels.append(int(line.split()[0]))
#                 print(label)
                
df=pd.DataFrame()
df["Labels"]=labels
df["Anatomical Regions"]=anats
df

,Labels,Anatomical Regions
0,0,Clear Label
1,1,corticofugal tract and corona radiata
2,3,Subthalamic nucleus
3,4,Molecular cell layer of the cerebellum
4,5,"Cerebellum, unspecified"
...,...,...
218,444,Temporal association cortex
219,448,"Secondary visual area, medial part"
220,500,Endopiriform nucleus
221,501,"Amygdaloid area, unspecified"


In [205]:
# Probe map for right-hemi implantation
siteMap = np.flip(np.reshape(scipy.io.loadmat('right_hemi_sitemap.mat')['siteMap'], -1))

# rEO_05 Electrode Map

## Elastic mapping

In [68]:
tpath="/Users/eminhanozil/electrode-localization/mri/rEO_05/output_Composite.h5"
movingPath="/Users/eminhanozil/electrode-localization/mri/rEO_05/sub-rEO_05_ses-presurgery_task-rest_acq-TurboRARE_T2w-ind_10_resampled.nii.gz"
fixedPath="/Users/eminhanozil/electrode-localization/mri/rEO_05/WHS_SD_rat_T2star_v1.01.nii.gz"
fixedcorr_path="/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/fixed_img-idx.npy"
movingcorr_path="/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/moving_presurgeryresampled_img-idx.npy"

fixed_coordinates, moving_coordinates = fixed2movingPoints(movingPath, fixedPath, tpath, fixedcorr_path, movingcorr_path)

In [175]:
#Bottom index at presurgery_resampled
idx=[165,185,304]
bottom_index =((moving_coordinates[:,0] == idx[0]) & (moving_coordinates[:,1] == idx[1]) & (moving_coordinates[:,2] == idx[2]))

#Top index at presurgery_resampled
idx=[170,271,312]
top_index =((moving_coordinates[:,0] == idx[0]) & (moving_coordinates[:,1] == idx[1]) & (moving_coordinates[:,2] == idx[2]))

In [180]:
atlas_path='/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/WHS_SD_rat_atlas_v4.nii.gz'
nii_atlas=nib.load(atlas_path)
atlas=np.asanyarray(nii_atlas.dataobj)
print(atlas.shape)

root='/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/'

fixed_coordinates=np.load(root+'fixed_img-idx.npy')
moving_coordinates=np.load(root+'moving_presurgeryresampled_img-idx.npy')

[[301. 578. 403.]]
[[306. 567. 317.]]
3.38694286931445
3.3741198852441507


In [207]:

electrode_trajectory=np.zeros(np.shape(atlas))
# bottom=fixed_coordinates[bottom_index]-1
# top=fixed_coordinates[top_index]-1
top=[170,271,312]
bottom=[165,185,304]
ch_coord,_=ch_coordinates(bottom, top, 64)

animal_id="rEO_05"
with open(root+animal_id+'-Elastic_chMap.txt', 'w') as f:
    for idx, coord in enumerate(ch_coord):
        atlasIdx=((moving_coordinates[:,0] == coord[0]) & (moving_coordinates[:,1] == coord[1]) & (moving_coordinates[:,2] == coord[2]))
        if fixed_coordinates[atlasIdx].any():
            atlasCoord=fixed_coordinates[atlasIdx][0]
            x,y,z =atlasCoord.astype(int)
        else: 
            print("no exact atlas coord")
        label=atlas[x,y,z]
        electrode_trajectory[x-1:x+1,y-1:y+1,z-1:z+1]=1
        anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
        line="CH:"+str(siteMap[2:][idx]-1)+" in "+anat_region+' Segment: '+str(label) + " atlas coord: " + str(atlasCoord)
        print(line)
        f.write(line)
        f.write('\n')
        
nii_trajectory=nib.Nifti1Image(electrode_trajectory, nii_atlas.affine)
nib.save(nii_trajectory, root+animal_id+'rEO_0_electrode_trajectory_segmentation.nii.gz')

[305 566 316]
CH:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 566 316]
[305 566 317]
CH:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 566 317]
[305 566 319]
CH:63 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 566 319]
[305 566 320]
CH:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 566 320]
[305 567 321]
CH:0 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 567 321]
[305 567 322]
CH:58 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 567 322]
[305 567 324]
CH:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 567 324]
[304 567 325]
CH:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [304 567 325]
[304 567 326]
CH:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: 

## Affine mapping

In [6]:
atlas_path='/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/rEO_05-affine-atlas.nii.gz'
nii_atlas=nib.load(atlas_path)
atlas=np.asanyarray(nii_atlas.dataobj)
print(atlas.shape)

In [32]:
#rEO-05 right hemi
# top=np.array([307, 572, 434])
# bottom=np.array([304, 565, 314])

top=np.array([170,271,312])
bottom=np.array([165,185,304])

ch_coord, unitvec=ch_coordinates(bottom, top, 64)
electrode_trajectory=np.zeros(np.shape(atlas))

In [34]:
animal_id="rEO_05"
with open(animal_id+'-Affine_chMap.txt', 'w') as f:
    for idx, coord in enumerate(ch_coord):
        print(coord)
        label=atlas[coord[0],coord[1],coord[2]]
        electrode_trajectory[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1,coord[2]-1:coord[2]+1]=1
        anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
        line="CH:"+str(siteMap[idx])+" in "+anat_region+' Segment: '+str(label) + " atlas coord: " + str(coord)
        print(line)
        f.write(line)
        f.write('\n')
    #     print('Segment: '+str(label))

[165 185 304]
CH:63 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 185 304]
[165 186 304]
CH:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 186 304]
[165 188 304]
CH:64 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 188 304]
[165 189 304]
CH:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 189 304]
[165 190 304]
CH:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 190 304]
[165 191 305]
CH:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 191 305]
[165 193 305]
CH:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 193 305]
[166 194 305]
CH:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [166 194 305]
[166 195 305]
CH:3 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 2

# rEO_06 Electrode Map

## Elastic mapping

In [230]:
root="/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_06/"
analysed="MRI_ANALYSED/elastic_map/"
transform="/Users/eminhanozil/Yanik Lab Dropbox/Eminhan Özil/Localization Manuscript 2024/RAT DATA/rEO_06/MRI_ANALYSED/elastic_map/s_register/output_Composite.h5"
fixed="/Users/eminhanozil/electrode-localization/mri/rEO_05/WHS_SD_rat_T2star_v1.01.nii.gz"
moving="MRI_RAW/bids/sub-rEO_06/ses-presurgery/anat/sub-rEO_06_ses-presurgery_task-rest_acq-TurboRAREregResampled39um_T2w-ind_00.nii.gz"
fixedPoints=root+analysed+"fixed_points.npy"
movingPoints=root+analysed+"moving_points.npy"
fixed_coordinates, moving_coordinates = fixed2movingPoints(movingPath=root+moving, fixedPath=fixed,transformPath=transform,fixedPointsSave=fixedPoints,movingPointsSave=movingPoints)

Succesfully loaded transform: output_Composite.h5
Succesfully loaded moving img: sub-rEO_06_ses-presurgery_task-rest_acq-TurboRAREregResampled39um_T2w-ind_00.nii.gz
Succesfully loaded fixed img: WHS_SD_rat_T2star_v1.01.nii.gz
Progress: 0/512
Progress: 10/512
Progress: 20/512
Progress: 30/512
Progress: 40/512
Progress: 50/512
Progress: 60/512
Progress: 70/512
Progress: 80/512
Progress: 90/512
Progress: 100/512
Progress: 110/512
Progress: 120/512
Progress: 130/512
Progress: 140/512
Progress: 150/512
Progress: 160/512
Progress: 170/512
Progress: 180/512
Progress: 190/512
Progress: 200/512
Progress: 210/512
Progress: 220/512
Progress: 230/512
Progress: 240/512
Progress: 250/512
Progress: 260/512
Progress: 270/512
Progress: 280/512
Progress: 290/512
Progress: 300/512
Progress: 310/512
Progress: 320/512
Progress: 330/512
Progress: 340/512
Progress: 350/512
Progress: 360/512
Progress: 370/512
Progress: 380/512
Progress: 390/512
Progress: 400/512
Progress: 410/512
Progress: 420/512
Progress: 4

In [247]:
#Top index at presurgery_resampled
idx=[176,254,303]
top_index =((moving_coordinates[:,0] == idx[0]) & (moving_coordinates[:,1] == idx[1]) & (moving_coordinates[:,2] == idx[2]))

#Bottom index at presurgery_resampled
idx=[168,191,310]
bottom_index =((moving_coordinates[:,0] == idx[0]) & (moving_coordinates[:,1] == idx[1]) & (moving_coordinates[:,2] == idx[2]))

j=0
print(fixed_coordinates[top_index][j])
print(fixed_coordinates[bottom_index][j])

print(np.linalg.norm(fixed_coordinates[top_index][j]-fixed_coordinates[bottom_index])*0.039)


[300. 552. 401.]
[310. 556. 334.]
2.646545862062473


In [278]:
atlas_path='/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/WHS_SD_rat_atlas_v4.nii.gz'
nii_atlas=nib.load(atlas_path)
atlas=np.asanyarray(nii_atlas.dataobj)
print(atlas.shape)

electrode_trajectory=np.zeros(np.shape(atlas))
# bottom=fixed_coordinates[bottom_index]-1
# top=fixed_coordinates[top_index][0]-1
top=np.array([176,254,303])
bottom=np.array([168,185,310])
ch_coord,_=ch_coordinates(bottom, top, 62)

root='/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_06/MRI_ANALYSED/elastic_map/'
animal_id="rEO_06"
with open(root+animal_id+'-Elastic_chMap.txt', 'w') as f:
    for idx, coord in enumerate(ch_coord):
        atlasIdx=((moving_coordinates[:,0] == coord[0]) & (moving_coordinates[:,1] == coord[1]) & (moving_coordinates[:,2] == coord[2]))
        if fixed_coordinates[atlasIdx].any():
            atlasCoord=fixed_coordinates[atlasIdx][0]
            x,y,z =atlasCoord.astype(int)
        else: 
            print("no exact atlas coord")
        label=atlas[x,y,z]
        electrode_trajectory[x-1:x+1,y-1:y+1,z-1:z+1]=1
        anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
        line="CH:"+str(siteMap[2:][idx]-1)+" in "+anat_region+' Segment: '+str(label) + " atlas coord: " + str(atlasCoord)
        print(line)
        f.write(line)
        f.write('\n')
        
nii_trajectory=nib.Nifti1Image(electrode_trajectory, nii_atlas.affine)
nib.save(nii_trajectory, root+'rEO_06_electrode_trajectory_segmentation.nii.gz')

(512, 1024, 512)
CH:63 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [310. 555. 329.]
CH:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [310. 555. 330.]
CH:0 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [310. 556. 331.]
CH:58 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [310. 556. 332.]
CH:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [309. 555. 333.]
CH:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [309. 555. 334.]
CH:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [309. 555. 336.]
CH:56 in Clear Label Segment: 0 atlas coord: [309. 555. 337.]
CH:3 in Clear Label Segment: 0 atlas coord: [309. 555. 338.]
CH:57 in Clear Label Segment: 0 atlas coord: [309. 555. 339.]
CH:4 in Dentate gyrus Segment: 96 atlas coord: [308. 555. 341.]
CH:54 in Dentate gyrus 

## Affine mapping

In [27]:
animal_id="rEO_06"
with open(animal_id+'-Affine_chMap.txt', 'w') as f:
    for idx, coord in enumerate(ch_coord):
        label=atlas[coord[0],coord[1],coord[2]]
        electrode_trajectory[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1,coord[2]-1:coord[2]+1]=1
        anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
        line="Site:"+str(siteMap[idx])+" in "+anat_region+' Segment: '+str(label)
        print(line)
        f.write(line)
        f.write('\n')
    #     print('Segment: '+str(label))

Site:63 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:64 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:3 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:57 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:4 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:58 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:5 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:55 in Latero

In [34]:
nii_trajectory=nib.Nifti1Image(electrode_trajectory, nii_atlas.affine)
nib.save(nii_trajectory, 'rEO_05_electrode_trajectory_segmentation.nii.gz')

In [58]:
eo06_20240201_bottom=np.array([108, 113, 200])
eo06_20240201_top=np.array([ 118, 182, 193])

ch_coord, unitvec=ch_coordinates(eo06_20240201_bottom, eo06_20240201_top, 64, 60)

In [31]:
atlas_path='/Users/mri_registration/SAMRI/samri_output/rEO_06/segmentation_rEO-06.nii.gz'
nii_eo06_atlas=nib.load(atlas_path)
eo06_atlas=np.asanyarray(nii_eo06_atlas.dataobj)

In [60]:
eo06_path='/Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-presurgery/anat/sub-rEO_06_ses-presurgery_task-rest_acq-TurboRAREreg_T2w-ind_00_resampled.nii.gz'
nii_eo06=nib.load(eo06_path)
eo06_img=np.asanyarray(nii_eo06.dataobj)

In [59]:
atlas=eo06_atlas
electrode_trajectory=np.zeros(np.shape(atlas))
for idx, coord in enumerate(ch_coord):
    label=atlas[coord[0],coord[1],coord[2]]
    electrode_trajectory[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1,coord[2]-1:coord[2]+1]=1
    anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
    print("Site:"+str(siteMap[idx])+" in "+anat_region+' Segment: '+str(label))
#     print('Segment: '+str(label))

Site:63 in Posterior thalamic nucleus Segment: 230.0
Site:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:64 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:3 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:57 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:4 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:58 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:5 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:55 in Laterodorsal thalamic nucleus

In [61]:
nii_trajectory=nib.Nifti1Image(electrode_trajectory, nii_eo06.affine)
nib.save(nii_trajectory, '/Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-postsurgery20240201/electrode_trajectory.nii.gz')